In [38]:
from utils import enviar_email, get_credentials, snowflake_login
from datetime import date
import os
import pandas as pd
import numpy as np

In [2]:
user, cursor, snow = snowflake_login()

Intento 1
Correct Password - connected to SNOWFLAKE


In [47]:
query = '''
WITH NA AS (
SELECT
    DISTINCT LGL.GEOG_LOCL_COD, FS.GEOG_LOCL_ID, FS.ARTC_ARTC_ID, 
FROM
    MSTRDB.DWH.FT_STOCK FS
JOIN
    MSTRDB.DWH.LU_GEOG_LOCAL LGL
ON
    LGL.GEOG_LOCL_ID = FS.GEOG_LOCL_ID AND LGL.GEOG_UNNG_ID = 2
WHERE
    FS.TIEM_DIA_ID = CURRENT_DATE - 2
AND
    FS.ARTC_ESTA_ID = 4
)

, NACT AS (
SELECT
    LGL.GEOG_LOCL_COD, FS.GEOG_LOCL_ID, LGL.GEOG_LOCL_DESC, FS.ARTC_ARTC_ID, FCUR.ORIGIN_COUNTRY_ID, FS.STCK_PRECIO_VENTA_DIA_CIVA AS PVP_ACTUAL,
    CASE WHEN FCUR.ORIGIN_COUNTRY_ID = 'UYU' THEN FCUR.COSTO_UNITARIO_RMS ELSE FS.STCK_COSTO_PROM_POND END AS COSTO_SIN_IVA,
    ROUND(DIV0(FS.STCK_PRECIO_VENTA_DIA_CIVA, FS.STCK_PRECIO_VENTA_DIA), 2) - 1 AS IVA, (COSTO_SIN_IVA * (IVA + 1)) AS COSTO_CON_IVA
FROM
    MSTRDB.DWH.FT_STOCK FS
JOIN
    MSTRDB.DWH.LU_GEOG_LOCAL LGL
ON
    LGL.GEOG_LOCL_ID = FS.GEOG_LOCL_ID AND LGL.GEOG_UNNG_ID = 2
LEFT JOIN
    NA
ON
    LGL.GEOG_LOCL_COD = NA.GEOG_LOCL_COD AND FS.ARTC_ARTC_ID = NA.ARTC_ARTC_ID
JOIN
    MSTRDB.DWH.FT_COSTO_UNITARIO_RMS FCUR
ON
    FS.GEOG_LOCL_ID = FCUR.GEOG_LOCL_ID AND FS.ARTC_ARTC_ID = FCUR.ARTC_ARTC_ID
    AND FCUR.TIEM_DIA_ID = (SELECT MAX(TIEM_DIA_ID) FROM MSTRDB.DWH.FT_COSTO_UNITARIO_RMS)
WHERE
    FS.TIEM_DIA_ID = CURRENT_DATE - 1
AND
    FS.ARTC_ESTA_ID = 4
AND
    NA.GEOG_LOCL_COD IS NULL
    )

, PRECIO_COMPETENCIA_LOC AS (
SELECT
    CAST(GEOG_LOCL_ID AS INT) AS GEOG_LOCL_ID, CAST(ARTC_ARTC_ID AS INT) AS ARTC_ARTC_ID, 'COMPETENCIA' AS DATO,
    ROUND(COMP_PREC_VENTA, 0) AS PRECIO, ROW_NUMBER() OVER (PARTITION BY GEOG_LOCL_ID, ARTC_ARTC_ID ORDER BY PRECIO ASC) RN
FROM
    SANDBOX_PLUS.DWH.LIMPIEZA_DATOS_DISPERSOS_COMPETENCIA
WHERE
    FLAG_DISPERSION_FINAL = '0'
AND
    FECHA_CHEQUEO = (SELECT MAX(FECHA_CHEQUEO) FROM SANDBOX_PLUS.DWH.LIMPIEZA_DATOS_DISPERSOS_COMPETENCIA)
QUALIFY
    RN = 1
)

, PRECIO_COMPETENCIA_NIL AS (
SELECT
    CAST(R.ARTC_ARTC_ID AS INT) AS ARTC_ARTC_ID, CAST(LOC.GEOG_LOCL_ID AS INT) AS GEOG_LOCL_ID, 'NIELSEN' AS DATO, ROUND(PRECIO, 0) AS PRECIO,
    ROW_NUMBER() OVER (PARTITION BY R.ARTC_ARTC_ID, LOC.GEOG_LOCL_ID ORDER BY NIELSEN.PRIORIDAD, NIELSEN.PRECIO) AS RN
FROM
    SANDBOX_PLUS.DWH.LIMPIEZA_DATOS_DISPERSOS_NIELSEN NIELSEN
JOIN
    MSTRDB.DWH.LU_ARTC_ARTICULO R
ON
    R.ORIN = NIELSEN.ORIN
JOIN
    MSTRDB.DWH.LU_GEOG_LOCAL LOC
ON
    LOC.GEOG_LOCL_COD = NIELSEN.GEOG_LOCL_COD
WHERE
    NIELSEN.PERIODO = (SELECT MAX(PERIODO) FROM SANDBOX_PLUS.DWH.LIMPIEZA_DATOS_DISPERSOS_NIELSEN)
AND
    NIELSEN.FLAG_DISPERSION_FINAL = '0'
QUALIFY
    RN = 1
)

, PRECIO_COMPETENCIA_RETAIL AS (
SELECT
    *, ROW_NUMBER() OVER (PARTITION BY ARTC_ARTC_ID, GEOG_LOCL_ID ORDER BY PRIORIDAD) AS RN_PRIORIDAD
FROM (
    SELECT 
        CAST(R.ARTC_ARTC_ID AS INT) AS ARTC_ARTC_ID, CAST(RETAIL.GEOG_LOCL_ID AS INT) AS GEOG_LOCL_ID,
        RETAIL.PRIORIDAD, 'RETAIL' AS DATO, MEDIAN(RETAIL.PRECIO_PRECIO) AS PRECIO,
        ROW_NUMBER() OVER (PARTITION BY R.ARTC_ARTC_ID, RETAIL.GEOG_LOCL_ID, RETAIL.NOMBRE_RETAIL ORDER BY RETAIL.TIEM_DIA_ID DESC) AS RN_FECHA
    FROM
        SANDBOX_PLUS.DWH.LIMPIEZA_DATOS_DISPERSOS_RETAIL RETAIL
    JOIN
        MSTRDB.DWH.LU_ARTC_ARTICULO R ON R.ORIN = RETAIL.ORIN
    WHERE
        PRIORIDAD IS NOT NULL
    AND
        RETAIL.FECHA_CHEQUEO = (SELECT MAX(FECHA_CHEQUEO) FROM SANDBOX_PLUS.DWH.LIMPIEZA_DATOS_DISPERSOS_RETAIL)
    AND
        RETAIL.FLAG_DISPERSION_FINAL = '0'
    GROUP BY 
        R.ARTC_ARTC_ID, RETAIL.GEOG_LOCL_ID, RETAIL.NOMBRE_RETAIL, RETAIL.TIEM_DIA_ID, RETAIL.PRIORIDAD
    QUALIFY
        RN_FECHA = 1) AS SUBQUERY
QUALIFY
    RN_PRIORIDAD = 1
)


, PRECIO_COMPETENCIA AS (
SELECT *
FROM (
    SELECT 
        ARTC_ARTC_ID, GEOG_LOCL_ID, DATO, PRECIO, 1 AS PRIORIDAD
    FROM
        PRECIO_COMPETENCIA_LOC
    UNION ALL
    SELECT 
        ARTC_ARTC_ID, GEOG_LOCL_ID, DATO, PRECIO, 2 AS PRIORIDAD
    FROM
        PRECIO_COMPETENCIA_RETAIL
    UNION ALL
    SELECT 
        ARTC_ARTC_ID, GEOG_LOCL_ID, DATO, PRECIO, 3 AS PRIORIDAD
    FROM
        PRECIO_COMPETENCIA_NIL
            ) AS FUENTES
QUALIFY ROW_NUMBER() OVER (PARTITION BY ARTC_ARTC_ID, GEOG_LOCL_ID ORDER BY PRIORIDAD) = 1
)

, TIENDA_RETAIL AS (
SELECT
    R.ARTC_ARTC_ID, RETAIL.GEOG_LOCL_ID, RETAIL.NOMBRE_RETAIL, RETAIL.TIEM_DIA_ID, MEDIAN(RETAIL.PRECIO_PRECIO) AS PRECIO,
    ROW_NUMBER() OVER (PARTITION BY R.ARTC_ARTC_ID, RETAIL.GEOG_LOCL_ID, RETAIL.NOMBRE_RETAIL ORDER BY RETAIL.TIEM_DIA_ID DESC) AS RN_FECHA
FROM
    SANDBOX_PLUS.DWH.LIMPIEZA_DATOS_DISPERSOS_RETAIL AS RETAIL
JOIN
    MSTRDB.DWH.LU_ARTC_ARTICULO AS R
ON
    R.ORIN = RETAIL.ORIN
WHERE
    PRIORIDAD IS NULL
AND
    RETAIL.FECHA_CHEQUEO = (SELECT MAX(FECHA_CHEQUEO) FROM SANDBOX_PLUS.DWH.LIMPIEZA_DATOS_DISPERSOS_RETAIL)
AND
    COALESCE(RETAIL.FLAG_DISPERSION_FINAL, 0) = '0'
GROUP BY
    1,2,3,4
QUALIFY
    RN_FECHA = 1
)

, TIENDA_NIELSEN AS (
SELECT
    DISTINCT R.ARTC_ARTC_ID, NIELSEN.STORE AS LOCAL_NIELSEN, LOC.GEOG_LOCL_ID, ROUND(NIELSEN.PRECIO, 0) AS PRECIO_NIELSEN, NIELSEN.PERIODO,
    ROW_NUMBER() OVER (PARTITION BY R.ARTC_ARTC_ID, LOC.GEOG_LOCL_ID ORDER BY NIELSEN.PRIORIDAD, NIELSEN.PRECIO) AS RN
FROM 
    SANDBOX_PLUS.DWH.LIMPIEZA_DATOS_DISPERSOS_NIELSEN AS NIELSEN
JOIN 
    MSTRDB.DWH.LU_ARTC_ARTICULO AS R 
ON
    R.ORIN = NIELSEN.ORIN
JOIN
    MSTRDB.DWH.LU_GEOG_LOCAL AS LOC 
ON
    LOC.GEOG_LOCL_COD = NIELSEN.GEOG_LOCL_COD
WHERE
    LOCAL_NIELSEN ILIKE '%TIENDA%ING%'
AND
    NIELSEN.FECHA_CHEQUEO = (SELECT MAX(FECHA_CHEQUEO) FROM SANDBOX_PLUS.DWH.LIMPIEZA_DATOS_DISPERSOS_NIELSEN)
AND
    NIELSEN.FLAG_DISPERSION_FINAL = '0'
QUALIFY
    RN = 1
)

, TIENDA_RESUMEN AS (
SELECT
    COALESCE(A.ARTC_ARTC_ID, B.ARTC_ARTC_ID) AS ARTC_ARTC_ID, COALESCE(A.GEOG_LOCL_ID, B.GEOG_LOCL_ID) AS GEOG_LOCL_ID,
    COALESCE(A.PRECIO, B.PRECIO_NIELSEN) AS PRECIO_TIENDA, COALESCE(NOMBRE_RETAIL, LOCAL_NIELSEN) AS LOCAL_TIENDA
FROM
    TIENDA_RETAIL AS A
FULL JOIN
    TIENDA_NIELSEN AS B
ON
    A.ARTC_ARTC_ID = B.ARTC_ARTC_ID AND A.GEOG_LOCL_ID = B.GEOG_LOCL_ID
)

, PRECIO_COMPETENCIA_MED AS (
SELECT ARTC_ARTC_ID, ROUND(MEDIAN(PRECIO)) AS PRECIO_MEDIANO_RELEVADO, 'MEDIANA RELEVADA' AS DATO
FROM
    PRECIO_COMPETENCIA
GROUP BY ALL
)

, MED AS (
SELECT
    FS.ARTC_ARTC_ID, MEDIAN(FS.STCK_PRECIO_VENTA_DIA_CIVA) AS PRECIO_MEDIANO_TATA, 'MEDIANA TATA' AS DATO
FROM
    MSTRDB.DWH.FT_STOCK FS
JOIN
    MSTRDB.DWH.LU_GEOG_LOCAL LGL
ON
    LGL.GEOG_LOCL_ID = FS.GEOG_LOCL_ID AND LGL.GEOG_UNNG_ID = 2
WHERE
    FS.TIEM_DIA_ID = CURRENT_DATE - 2
AND
    FS.ARTC_ESTA_ID = 4
GROUP BY ALL
)

, F AS (
SELECT
    F.GRUPO AS GRUPO_T, NACT.GEOG_LOCL_COD AS CODIGO_TIENDA, NACT.GEOG_LOCL_DESC AS NOMBRE_TIENDA, G.GROUP_NAME AS GRUPO, D.DEPT_NAME AS DEPT,
    C.CLASS_NAME AS CLASS, S.SUB_NAME AS SUBLCLASS, FAM.FAMILIA_FINAL AS FAMILIA, FAM.INTEGRANTES_FAMILIA, LAA.ORIN, LAA.ARTC_ARTC_DESC,
    NACT.COSTO_CON_IVA, NACT.PVP_ACTUAL, ROUND(DIV0(NACT.PVP_ACTUAL - NACT.COSTO_CON_IVA, NACT.PVP_ACTUAL), 2) AS MG_ACTUAL, CAN.CANASTA, NACT.IVA,
    NACT.ORIGIN_COUNTRY_ID AS CURRENCY_CODE, LOC.PRECIO AS PRECIO_COMPETENCIA_LOCALES, R.PRECIO AS PRECIO_RETAIL, N.PRECIO AS PRECIO_NIELSEN,
    PCM.PRECIO_MEDIANO_RELEVADO, MED.PRECIO_MEDIANO_TATA, ROUND(((1 + MG.MG) * NACT.COSTO_CON_IVA)) AS PRECIO_MARGEN_OBJETIVO,
    ROUND(TR.PRECIO_TIENDA) AS PRECIO_TIENDA_INGLESA,
    ROUND(COALESCE(PC.PRECIO, PCM.PRECIO_MEDIANO_RELEVADO, MED.PRECIO_MEDIANO_TATA, (1 + MG.MG) * NACT.COSTO_CON_IVA, NULL)) AS PVP_NUEVO,
    COALESCE(PC.DATO, PCM.DATO, MED.DATO, CASE WHEN MG.MG IS NOT NULL THEN 'MARGEN OBJETIVO' ELSE NULL END) AS DATO,
    COALESCE(REM.EXCLUIDO_COMERCIAL, 0) AS EXCLUIDO_COMERCIAL, COALESCE(REM.MARCA_PROPIA, 0) AS MARCA_PROPIA,
    REM.RAZON_DE_EXCLUSION, COALESCE(REM.ESTA_EN_PROMO, 0) AS ESTA_EN_PROMO, COALESCE(REM.ESTA_EN_LIQUIDACION, 0) AS ESTA_EN_LIQUIDACION
FROM
    NACT
JOIN
    MSTRDB.DWH.LU_ARTC_ARTICULO LAA
ON
    LAA.ARTC_ARTC_ID = NACT.ARTC_ARTC_ID
JOIN
    MSTRDB.DWH.ITEM_MASTER IM
ON
    IM.ITEM = LAA.ORIN
JOIN
    MSTRDB.DWH.DEPS D
ON
    D.DEPT = IM.DEPT
JOIN
    MSTRDB.DWH.GROUPS G
ON
    G.GROUP_NO = D.GROUP_NO AND G.GROUP_NO IN (1, 3, 6, 25, 9, 11, 12, 4, 14, 15, 18, 16, 42)
JOIN
    MSTRDB.DWH.CLASS C
ON
    C.CLASE = IM.CLASE
JOIN
    MSTRDB.DWH.SUBCLASS S
ON
    S.SUBCLASE = IM.SUBCLASE
JOIN
    SANDBOX_PLUS.DWH.FORMATO_LOCALES_CON_LCL_ID F
ON
    F.GEOG_LOCL_ID = NACT.GEOG_LOCL_ID
LEFT JOIN
    PRECIO_COMPETENCIA PC
ON
    PC.ARTC_ARTC_ID = NACT.ARTC_ARTC_ID AND PC.GEOG_LOCL_ID = NACT.GEOG_LOCL_ID
LEFT JOIN
    PRECIO_COMPETENCIA_LOC LOC
ON
    LOC.ARTC_ARTC_ID = NACT.ARTC_ARTC_ID AND LOC.GEOG_LOCL_ID = NACT.GEOG_LOCL_ID
LEFT JOIN
    PRECIO_COMPETENCIA_NIL N
ON
    N.ARTC_ARTC_ID = NACT.ARTC_ARTC_ID AND N.GEOG_LOCL_ID = NACT.GEOG_LOCL_ID
LEFT JOIN
    PRECIO_COMPETENCIA_RETAIL R
ON
    R.ARTC_ARTC_ID = NACT.ARTC_ARTC_ID AND R.GEOG_LOCL_ID = NACT.GEOG_LOCL_ID
LEFT JOIN
    TIENDA_RESUMEN TR
ON
    TR.ARTC_ARTC_ID = NACT.ARTC_ARTC_ID AND TR.GEOG_LOCL_ID = NACT.GEOG_LOCL_ID
LEFT JOIN
    PRECIO_COMPETENCIA_MED PCM
ON
    PCM.ARTC_ARTC_ID = NACT.ARTC_ARTC_ID
LEFT JOIN
    MED
ON
    MED.ARTC_ARTC_ID = NACT.ARTC_ARTC_ID
LEFT JOIN
    SANDBOX_PLUS.DWH.INPUT_PRICING_MARGEN_OBJETIVO AS MG
ON
    MG.ORIN = LAA.ORIN
LEFT JOIN
    (
    SELECT
        DISTINCT ITEM, FAMILIA_FINAL, COUNT(*) OVER (PARTITION BY FAMILIA_FINAL) AS INTEGRANTES_FAMILIA
    FROM
        SANDBOX_PLUS.DWH.FAMILIAS_DE_PRODUCTOS
    ) AS FAM
ON
    FAM.ITEM = LAA.ORIN
LEFT JOIN
    SPIKE.SPIKE.CANASTAS CAN
ON
    CAN.ITEM = IM.ITEM
LEFT JOIN
    SANDBOX_PLUS.DWH.REGISTROS_ENVIO_DE_INFO_MARCELO_SEMANAL REM
ON
    REM.CODIGO_TIENDA = NACT.GEOG_LOCL_COD AND REM.ORIN = LAA.ORIN
    AND REM.FECHA_ENVIO_INFO = (SELECT MAX(FECHA_ENVIO_INFO) FROM SANDBOX_PLUS.DWH.REGISTROS_ENVIO_DE_INFO_MARCELO_SEMANAL)
)

SELECT
    GRUPO_T, CODIGO_TIENDA, NOMBRE_TIENDA, GRUPO, DEPT, CLASS, SUBLCLASS, FAMILIA, INTEGRANTES_FAMILIA, ORIN, ARTC_ARTC_DESC, CANASTA, IVA,
    COSTO_CON_IVA, PVP_ACTUAL, MG_ACTUAL, PRECIO_COMPETENCIA_LOCALES, PRECIO_RETAIL, PRECIO_NIELSEN, PRECIO_MEDIANO_RELEVADO, PRECIO_MEDIANO_TATA,
    PRECIO_MARGEN_OBJETIVO, PRECIO_TIENDA_INGLESA, PVP_NUEVO, DATO, ROUND(DIV0(PVP_NUEVO - COSTO_CON_IVA, PVP_NUEVO), 2) AS MG_NUEVO,
    
    CASE WHEN PVP_NUEVO > PVP_ACTUAL THEN 'SUBE' WHEN PVP_NUEVO < PVP_ACTUAL THEN 'BAJA' WHEN PVP_NUEVO = PVP_ACTUAL THEN 'SE MANTIENE'
    ELSE 'OTRO' END AS ACCION, PVP_NUEVO - PVP_ACTUAL AS DIF_$, DIV0((PVP_NUEVO - PVP_ACTUAL), PVP_ACTUAL) AS DIF_PORC,
    
    ROUND(EXCLUIDO_COMERCIAL) AS EXCLUIDO_COMERCIAL, ROUND(MARCA_PROPIA) AS MARCA_PROPIA, RAZON_DE_EXCLUSION,
    ROUND(ESTA_EN_PROMO) AS ESTA_EN_PROMO, ROUND(ESTA_EN_LIQUIDACION) AS ESTA_EN_LIQ, CURRENCY_CODE, CURRENT_DATE AS FECHA_ENVIO
FROM
    F;
'''

In [48]:
cursor.execute(query)
df = cursor.fetch_pandas_all()
df

,GRUPO_T,CODIGO_TIENDA,NOMBRE_TIENDA,GRUPO,DEPT,CLASS,SUBLCLASS,FAMILIA,INTEGRANTES_FAMILIA,ORIN,...,ACCION,DIF_$,DIF_PORC,EXCLUIDO_COMERCIAL,MARCA_PROPIA,RAZON_DE_EXCLUSION,ESTA_EN_PROMO,ESTA_EN_LIQ,CURRENCY_CODE,FECHA_ENVIO
0,CERCANIA-AAA,320,Express Cavia,BEBIDAS,BEB. NO ALCOHOLICAS,AGUAS,AGUA SIN GAS,1000417713,1,1000417713,...,BAJA,-4.0,-0.032258,0,0,None,0,0,UYU,2025-09-17
1,CERCANIA-AAA,411,Express Rivera,BEBIDAS,BEB. NO ALCOHOLICAS,AGUAS,AGUA SIN GAS,1000417713,1,1000417713,...,BAJA,-9.0,-0.072581,0,0,None,0,0,UYU,2025-09-17
2,SUPER-AAA,103,Super Centro,TEXTIL,HOMBRE,VESTIMENTA,REMERAS MANGA CORTA,1000479989,6,1000480228,...,OTRO,NaN,NaN,0,0,None,0,0,USD,2025-09-17
3,HIPER-AAA,115,Hiper Tres Cruces,TEXTIL,HOMBRE,VESTIMENTA,REMERAS MANGA CORTA,1000479989,6,1000480227,...,OTRO,NaN,NaN,0,0,None,0,0,USD,2025-09-17
4,SUPER-AAA,159,Super Shopping Paysandú,TEXTIL,HOMBRE,INTERIORES,BOXERS,1000020127,1,1000020127,...,OTRO,NaN,NaN,0,0,None,0,0,USD,2025-09-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1517,HIPER-AAA,143,Hiper Artigas,COMESTIBLES,GOLOSINAS,BOMBONES,BOCADITOS DULCES,1000570093,1,1000570093,...,OTRO,NaN,NaN,0,0,None,0,0,UYU,2025-09-17
1518,HIPER-AAA,126,Hiper Tacuarembo,COMESTIBLES,GOLOSINAS,BOMBONES,BOCADITOS DULCES,1000570093,1,1000570093,...,OTRO,NaN,NaN,0,0,None,0,0,UYU,2025-09-17
1519,SUPER-AA,167,Hiper MAC,TEXTIL,HOMBRE,VESTIMENTA,REMERAS MANGA CORTA,1000479989,6,1000480231,...,OTRO,NaN,NaN,0,0,None,0,0,USD,2025-09-17
1520,SUPER-AAA,159,Super Shopping Paysandú,TEXTIL,HOMBRE,VESTIMENTA,CAMISAS MANGA CORTA,1000458189,6,1000458957,...,OTRO,NaN,NaN,0,0,None,0,0,USD,2025-09-17


In [10]:
df_dm = pd.read_excel('Libro1.xlsx').astype('str')
df_dm

,ITEM,LOC
0,1000056844,420
1,1000020561,159
2,1000520682,304
3,1000369408,117
4,1000369408,119
...,...,...
1603,1000570128,323
1604,1000570128,327
1605,1000570128,328
1606,1000570128,331


In [49]:
df_c = df[['ORIN', 'CODIGO_TIENDA']].drop_duplicates().astype('str')
df_c.columns = df_dm.columns
df_c

,ITEM,LOC
0,1000417713,320
1,1000417713,411
2,1000480228,103
3,1000480227,115
4,1000020127,159
...,...,...
1517,1000570093,143
1518,1000570093,126
1519,1000480231,167
1520,1000458957,159


In [51]:
prueba = df_dm.merge(df_c, how='left', indicator=True)
prueba

,ITEM,LOC,_merge
0,1000056844,420,left_only
1,1000020561,159,both
2,1000520682,304,left_only
3,1000369408,117,both
4,1000369408,119,both
...,...,...,...
1603,1000570128,323,both
1604,1000570128,327,both
1605,1000570128,328,both
1606,1000570128,331,both


In [56]:
prueba[(prueba['_merge']!='both')]['ITEM'].unique()

array(['1000056844', '1000520682', '1000555615', '1000038946',
       '1000460492', '1000037679', '1000495557', '1000347944',
       '1000349560', '1000570090', '1000570095', '1000057759',
       '1000056556'], dtype=object)

In [44]:
prueba[(prueba['_merge']!='both')]['LOC'].value_counts()

LOC
307    6
198    5
304    2
166    1
310    1
      ..
143    1
140    1
139    1
134    1
334    1
Name: count, Length: 76, dtype: int64

In [57]:
prueba[(prueba['_merge']!='both') &
       (prueba['ITEM'].isin(['1000056844', '1000520682', '1000555615', '1000038946', '1000460492',
                             '1000037679', '1000495557', '1000347944', '1000349560', '1000570095',
                             '1000057759', '1000056556']))]

,ITEM,LOC,_merge
0,1000056844,420,left_only
2,1000520682,304,left_only
70,1000555615,198,left_only
71,1000038946,198,left_only
146,1000460492,198,left_only
147,1000037679,307,left_only
227,1000495557,307,left_only
253,1000347944,307,left_only
942,1000349560,307,left_only
1018,1000570095,198,left_only


In [61]:
date.today().year

2025

In [62]:
if len(str(date.today().month)) > 1:
    mes = str(date.today().month)
else:
    mes = '0' + str(date.today().month)
mes

'09'

In [63]:
if len(str(date.today().day)) > 1:
    dia = str(date.today().day)
else:
    dia = '0' + str(date.today().day)
dia

'17'

In [64]:
df.to_excel('NUEVAS ACTIVACIONES ' + str(date.today().year) + mes + dia + '.xlsx', index=False)

In [65]:
enviar_email(sender='marcos.larran@tata.com.uy', receiver=['marcela.moreira@tata.com.uy', 'nahuel.hartwig@tata.com.uy'],
             subject='Nuevas activaciones',
             body='Buenas tardes\n\nSe envían las nuevas activaciones del día de ayer.\n\nSaludos,',
             files=['NUEVAS ACTIVACIONES ' + str(date.today().year) + mes + dia + '.xlsx'])

Correo enviado exitosamente.


True

In [66]:
os.remove('NUEVAS ACTIVACIONES ' + str(date.today().year) + mes + dia + '.xlsx')

In [ ]:
enviar_email(sender='marcos.larran@tata.com.uy', receiver=['ds-team@gdn.com.uy'],
             subject='Nuevas activaciones', body='Hubo un problema con el flujo', files=[])